Preprocessing, Model Selection and Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.utils import Sequence
import cv2
import os
import joblib
from sklearn.utils.class_weight import compute_class_weight

num_classes = 8  

# Loading the InceptionV3 model pre-trained on ImageNet, excluding the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) 
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing/unfreezing the layers of the base model for fine tuning
for layer in base_model.layers[:-50]:
    layer.trainable = False

initial_learning_rate = 0.0001
model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Reading the CSV file
data_df = pd.read_csv(r'D:\Academics\IIT\Tech\COSMOSOC\SOI-sds\train_dataset\train.csv')
image_dir = r'D:\Academics\IIT\Tech\COSMOSOC\SOI-sds\train_dataset\train_dataset'

image_files = os.listdir(image_dir)
image_files = [os.path.splitext(filename)[0] for filename in image_files]

# Filter the DataFrame to include only existing image filenames
data_df = data_df[data_df['File_Name'].apply(lambda x: os.path.splitext(x)[0] in image_files)]

data_df.to_csv(r'D:\Academics\IIT\Tech\COSMOSOC\SOI-sds\train_dataset\train.csv', index=False)

label_encoder = LabelEncoder()
data_df['label'] = label_encoder.fit_transform(data_df['label'])

num_classes = len(label_encoder.classes_)

train_df, val_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Custom data generator(Data augmentation part)
class CustomDataGenerator(Sequence):
    def __init__(self, df, batch_size, image_size, image_dir, num_classes, augment=True):
        self.df = df
        self.batch_size = batch_size
        self.image_size = image_size
        self.image_dir = image_dir
        self.num_classes = num_classes
        self.augment = augment
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()
        self.datagen = ImageDataGenerator(
            rescale=1.0/255.0,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1
        ) if self.augment else ImageDataGenerator(rescale=1.0/255.0)

    def __len__(self):
        return len(self.df) // self.batch_size

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        images = np.empty((self.batch_size, *self.image_size, 3), dtype=np.float32)
        labels = np.empty((self.batch_size, self.num_classes), dtype=np.float32)

        for i, (_, row) in enumerate(batch_df.iterrows()):
            image_path = os.path.join(self.image_dir, row['File_Name'])
            image = cv2.imread(image_path)
            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping...")
                continue
            image = cv2.resize(image, self.image_size)
            if self.augment:
                image = self.datagen.random_transform(image)
            images[i] = image
            labels[i] = tf.keras.utils.to_categorical(row['label'], num_classes=self.num_classes)

        return images, labels


batch_size = 16
image_size = (224, 224)


train_generator = CustomDataGenerator(train_df, batch_size, image_size, image_dir, num_classes, augment=True)
val_generator = CustomDataGenerator(val_df, batch_size, image_size, image_dir, num_classes, augment=False)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

class_weights = dict(enumerate(class_weights))

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(0.01))  

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True),
    tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
]

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15, 
    callbacks=callbacks,
    class_weight=class_weights
)

# Save the label encoder for future use
joblib.dump(label_encoder, 'label_encoder.pkl')



Uploading the test dataset to the model and obtaining predicted labels

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
import joblib

test_image_dir = r'D:\Academics\IIT\Tech\COSMOSOC\SOI-sds\test_dataset\test_dataset'
image_size = (224, 224)
model = tf.keras.models.load_model('best_model.keras')

label_encoder = joblib.load('label_encoder.pkl')

test_image_files = [f for f in os.listdir(test_image_dir) if os.path.isfile(os.path.join(test_image_dir, f))]

# Preprocessing of the test images
test_images = []
image_names = []

for filename in test_image_files:
    image_path = os.path.join(test_image_dir, filename)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image {image_path}. Skipping...")
        continue
    image = cv2.resize(image, image_size)
    image = image.astype(float) / 255.0
    test_images.append(image)
    image_names.append(filename)


test_images = np.array(test_images)
print("Test images shape:", test_images.shape)

# Predicting labels in batches to avoid memory issues
batch_size = 16
num_batches = int(np.ceil(len(test_images) / batch_size))

all_predictions = []

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(test_images))
    batch_images = test_images[start_idx:end_idx]
    batch_predictions = model.predict(batch_images)
    all_predictions.append(batch_predictions)


all_predictions = np.concatenate(all_predictions, axis=0)

predicted_labels = np.argmax(all_predictions, axis=1)

# Decoding the numerical labels to categorical class names
decoded_labels = label_encoder.inverse_transform(predicted_labels)

output_df = pd.DataFrame({'File_Name': image_names, 'Predicted_Label': decoded_labels})
output_df.to_csv('test_predictions.csv', index=False)
print('Predicted labels stoered in the test_predictions.csv file sucessfuly!')
